In [1]:
%load_ext autoreload
%autoreload 2
from infovis21.datamodel.Track import Track
from infovis21.datamodel.User import User
from infovis21.mongodb import MongoAccess as ma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
# # I know the without is a bit redundant as we could just use the total size of the collection which would be faster but it's also a sanity check for the edge case values of genres

# pipeline_with_genre = [
#     {'$set': {'n_of_genres': {'$size' : {'$ifNull': [ "$genres", [] ]}} } },
#     {'$match': {'n_of_genres': {'$gt': 0}}}
# ]
# res_with_genre = list(ma.coll_artists.aggregate(pipeline_with_genre))
# print(f'With genre info in artists {len(res_with_genre)}')

# pipeline_without_genre = [
#     {'$set': {'n_of_genres': {'$size' : {'$ifNull': [ "$genres", [] ]}} } },
#     {'$match': {'n_of_genres': {'$eq': 0}}}
# ]
# res_without_genre = list(ma.coll_artists.aggregate(pipeline_without_genre))
# print(f'Without genre info in artists {len(res_without_genre)}')

# res_with_genre = list(ma.coll_tracks.aggregate(pipeline_with_genre))
# print(f'With genre info in tracks {len(res_with_genre)}')
# res_without_genre = list(ma.coll_tracks.aggregate(pipeline_without_genre))
# print(f'Without genre info in tracks {len(res_without_genre)}')


df_genres = pd.DataFrame(ma.get_collection(ma.coll_genres))

In [21]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import minmax_scale
import os


# from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score

# dataframe = pd.read_csv("/Users/simontokloth/Documents/data/data_by_genres.csv")

df = df_genres.drop(['labels', 'preview_url', 'id', 'name', 'genres', 'duration_ms', 'popularity'], axis=1)

In [22]:
from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=11, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm='auto')
Kmean.fit(df)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:788: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  ". It has no effect", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)


KMeans(n_clusters=11, n_jobs=None, precompute_distances='auto')

In [23]:
df['label'] = pd.DataFrame(Kmean.labels_)

df = pd.concat([df_genres.genres, df],axis=1)

In [24]:
# to check what's in each bin

df.loc[df['label'] == 0]

,genres,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,acousticness,year,label
31,[cumbia ecuatoriana],0.566444,0.339222,0.442653,0.236000,-12.496278,0.064106,134.320000,0.666056,5.777778,0.277778,0.862167,1939.722222,0
85,[german punk],0.417475,0.846050,0.061627,0.268330,-6.997525,0.089012,143.744150,0.581025,5.525000,0.725000,0.071864,1947.550000,0
122,[brazilian ska],0.320000,0.991000,0.000004,0.226000,-7.047000,0.220000,167.532000,0.545000,9.000000,0.000000,0.500000,1940.000000,0
126,[historic string quartet],0.333427,0.134927,0.532698,0.155499,-18.905971,0.046117,114.366699,0.294399,5.048544,0.786408,0.949262,1946.281553,0
179,[traditional rockabilly],0.650214,0.790857,0.002003,0.147450,-5.232429,0.087886,118.516714,0.864786,5.571429,0.857143,0.576786,1956.285714,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3144,[western swing],0.574915,0.344018,0.082146,0.175397,-11.927194,0.060170,120.167848,0.641315,4.957576,0.896970,0.802116,1959.921212,0
3209,[ghanaian traditional],0.644500,0.077200,0.000000,0.149000,-19.035500,0.492500,145.207500,0.514000,0.500000,1.000000,0.761000,1954.000000,0
3216,[vintage tango],0.630396,0.268175,0.471062,0.197549,-13.763390,0.113332,119.924801,0.718859,5.156186,0.661654,0.982193,1931.388585,0
3226,[delta blues],0.592301,0.362307,0.096589,0.186486,-12.220561,0.121798,118.746402,0.685866,5.638037,0.832822,0.834701,1949.263804,0


In [25]:
# labeling of main genre based on genres 

df.label = df.label.replace(0, "Pop")

df.label = df.label.replace(1, "Classical")
 
df.label = df.label.replace(2, "Jazz")

df.label = df.label.replace(3, "Reggae")

df.label = df.label.replace(4, "Country")

df.label = df.label.replace(5, "Rock")

df.label = df.label.replace(6, "Electronic")

df.label = df.label.replace(7, "Blues")

df.label = df.label.replace(8, "Electronic")

df.label = df.label.replace(9, "Hiphop")

df.label = df.label.replace(10, "Classical")

,genres,genres,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,acousticness,year,label
6,[slovak hip hop],[slovak hip hop],0.606000,0.898857,0.000000,0.274857,-3.890429,0.274286,100.068571,0.559429,5.714286,0.428571,0.216643,2014.000000,0
15,[finnish alternative rock],[finnish alternative rock],0.600000,0.796000,0.000052,0.484000,-4.481000,0.027500,105.991000,0.750000,2.000000,1.000000,0.000109,2003.000000,0
21,[k-rock],[k-rock],0.598565,0.519283,0.016310,0.208074,-7.607348,0.060657,113.300043,0.446348,4.782609,0.695652,0.514739,2015.739130,0
24,[progressive power metal],[progressive power metal],0.223000,0.885000,0.001050,0.243000,-3.687000,0.057000,101.795000,0.143000,9.000000,0.000000,0.000031,2002.000000,0
29,[cambridgeshire indie],[cambridgeshire indie],0.358000,0.546000,0.058600,0.111000,-8.927000,0.029600,106.489000,0.264000,0.000000,0.000000,0.795000,2014.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,[ghanaian pop],[ghanaian pop],0.940000,0.518000,0.000000,0.287000,-8.484000,0.373000,107.075000,0.932000,5.000000,0.000000,0.194000,2014.000000,0
3193,[west coast trap],[west coast trap],0.766824,0.644387,0.010972,0.218118,-7.131471,0.244214,104.641790,0.538127,5.092437,0.621849,0.106240,2004.605042,0
3201,[rap napoletano],[rap napoletano],0.674000,0.603000,0.000000,0.094100,-13.198000,0.055900,109.960000,0.404000,0.000000,0.000000,0.198000,2018.000000,0
3228,[modern southern rock],[modern southern rock],0.535000,0.644220,0.054428,0.155200,-6.717488,0.037885,112.743780,0.458024,5.682927,0.707317,0.135660,2008.536585,0


In [29]:
def label_color(df):
    if df['label'] == "Classical" :
        return '#212F3D'
    if df['label'] == "Electronic":
        return '#FF5733'
    if df['label'] == "Jazz" :
        return '#FFE033'
    if df['label'] == "Hiphop":
        return 'color'
    if df['label'] == "Blues":
        return 'color'
    if df['label'] == "Rock":
        return 'color'
    if df['label'] == "Pop":
        return 'color'
    if df['label'] == "Country":
        return 'color'
    if df['label'] == "Reggae":
        return '#D733F'

In [30]:
df['label_color'] = df.apply(label_color, axis=1)